In [1]:
!mkdir -p ~/.kaggle
!cp /kaggle/input/username/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle --version

Kaggle API 1.7.4.5


In [3]:
# ==============================================
# Kaggle pix2pix (Canny->RGB) with external datasets
# - Frequent checkpoint versions to a dedicated dataset
# - Samples/history versions to a sepגם'arate dataset
# - Auto resume from latest external checkpoint
# ==============================================

# -------- USER CONFIG: CHANGE THESE ---------------
INPUT_DATASET           = "yealsh/canny2segmentation-dataset"    # dataset with your train/val (no test needed yet)
KAGGLE_SECRET_DATASET   = "yealsh/kaggle-json-secret"            # dataset containing kaggle.json
CHECKPOINTS_DATASET_SLUG= "edgeseg-pix2pix-ckpts"                # external dataset for frequent checkpoints
OUTPUTS_DATASET_SLUG    = "edgeseg-pix2pix-outputs"              # external dataset for samples/history

IMG_SIZE   = 256
BATCH_SIZE = 8
MAX_EPOCHS = 200
LR_G = 1e-4
LR_D = 1e-4
LAMBDA_L1 = 150
SEED = 42

# version cadence
EXPORT_CKPT_EVERY_STEPS = 200     # גרס צ'קפוינט כל 200 צעדים
EXPORT_SAMPLES_EVERY_E  = 5       # גרס דוגמאות/היסטוריה כל 5 אפוקים

# -------------------------------------------------
import os, json, glob, shutil, random, time, re
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# ---------- Kaggle API: auth from secret dataset ----------
os.makedirs("/root/.kaggle", exist_ok=True)
secret_candidates = glob.glob("/kaggle/input/*/kaggle.json")
if not secret_candidates:
    raise RuntimeError("לא נמצא kaggle.json ב-Input. הוסיפי את הדאטהסט עם kaggle.json ל-Notebook (Add data).")
shutil.copy(secret_candidates[0], "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)

# small check
!kaggle --version

# -------- derive USERNAME from kaggle.json (IMPORTANT!) --------
with open("/root/.kaggle/kaggle.json") as f:
    _kg = json.load(f)
USERNAME = _kg.get("username") or _kg.get("user")
if not USERNAME:
    raise RuntimeError("לא הצלחתי לקרוא username מתוך kaggle.json")

# -------- sanitize slugs just in case --------
def _sanitize_slug(s):
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9\-]", "-", s)
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s
CHECKPOINTS_DATASET_SLUG = _sanitize_slug(CHECKPOINTS_DATASET_SLUG)
OUTPUTS_DATASET_SLUG    = _sanitize_slug(OUTPUTS_DATASET_SLUG)

# -------- Paths --------
WORK_DIR = "/kaggle/working"
CKPT_LOCAL_DIR = f"{WORK_DIR}/CKPTS"     # temp local stage for ckpts
OUT_LOCAL_DIR  = f"{WORK_DIR}/EXPORT"    # temp local stage for samples/history
os.makedirs(CKPT_LOCAL_DIR, exist_ok=True)
os.makedirs(OUT_LOCAL_DIR,  exist_ok=True)

# -------- utils --------
def set_seed(s=SEED):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed()

def download_latest_pair_only(username, slug, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    # קבלי רשימת קבצים (ייתכן פאג'ינציה, לכן לולאה עם page-token)
    token = None
    files = []
    while True:
        cmd = f'kaggle datasets files {username}/{slug}'
        if token: cmd += f' --page-token {token}'
        out = os.popen(cmd).read()
        files += re.findall(r'(G_epoch_\d+\.pt|D_epoch_\d+\.pt)', out)
        m = re.search(r'Next Page Token = (\S+)', out)
        if m: token = m.group(1)
        else: break

    if not files:
        print("[INFO] אין קבצים בדאטהסט הצ׳קפוינטים — נתחיל מאפס.")
        return

    # מצאי האפוק המקסימלי שיש לו גם G וגם D
    g_epochs = {int(re.search(r'_(\d+)\.pt$', f).group(1)) for f in files if f.startswith('G_epoch_')}
    d_epochs = {int(re.search(r'_(\d+)\.pt$', f).group(1)) for f in files if f.startswith('D_epoch_')}
    common = sorted(g_epochs & d_epochs)
    if not common:
        print("[WARN] נמצאו קבצים אך לא זוגות G/D תואמים.")
        return
    e = common[-1]
    need = [f"G_epoch_{e:03d}.pt", f"D_epoch_{e:03d}.pt"]

    # הורדת שני הקבצים בלבד
    for fname in need:
        os.system(f'kaggle datasets download -d {username}/{slug} -f {fname} -p "{target_dir}" -q')
    # הם יורדים כ-zip; נחלץ אם צריך
    for z in glob.glob(os.path.join(target_dir, "*.zip")):
        os.system(f'unzip -o "{z}" -d "{target_dir}" >/dev/null')
        os.remove(z)
    print(f"[OK] הורדתי רק את {need}")

def list_pairs(split, allow_empty=False):
    """
    pairs = [(edge_path, overlay_path)] for split ∈ {train,val,test}
    Supports:
      - SEM_edges_2040/**  <->  SEM_overlay_2040/**   (train)
      - val_edges/**       <->  val_overlay/**        (val)
      - test_edges/**      <->  test_overlay/**       (test optional)
    """
    if split == "train":
        E_dirname, O_dirname = "SEM_edges_2040", "SEM_overlay_2040"
    elif split == "val":
        E_dirname, O_dirname = "val_edges", "val_overlay"
    elif split == "test":
        E_dirname, O_dirname = "test_edges", "test_overlay"
    else:
        raise ValueError("split must be train/val/test")

    roots = sorted([p for p in glob.glob("/kaggle/input/*") if os.path.isdir(p)])

    edge_files, over_files = [], []
    for root in roots:
        e_root = os.path.join(root, E_dirname)
        o_root = os.path.join(root, O_dirname)
        if os.path.isdir(e_root):
            edge_files += glob.glob(os.path.join(e_root, "**", "*_edges.*"), recursive=True)
        if os.path.isdir(o_root):
            over_files += glob.glob(os.path.join(o_root, "**", "*_overlay.*"), recursive=True)

    if (not edge_files or not over_files):
        if allow_empty:
            return []
        else:
            raise RuntimeError(
                f"Didn't find expected folders/files for split='{split}'. "
                f"Looked for '{E_dirname}' and '{O_dirname}' under /kaggle/input/*"
            )

    def base_key(p, suffix):
        name = os.path.splitext(os.path.basename(p))[0]
        return name.replace(suffix, "")

    edges_by_key = {base_key(p, "_edges"): p for p in edge_files}
    over_by_key  = {base_key(p, "_overlay"): p for p in over_files}

    keys = sorted(set(edges_by_key) & set(over_by_key))
    pairs = [(edges_by_key[k], over_by_key[k]) for k in keys]

    if not pairs and not allow_empty:
        raise RuntimeError(
            f"No matching pairs for split='{split}'. "
            "Make sure filenames share the same base (e.g., *_edges ↔ *_overlay)."
        )
    return pairs

class PairDataset(Dataset):
    def __init__(self, pairs, size=256):
        self.items = pairs; self.size = size
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        a_path, b_path = self.items[i]
        A = Image.open(a_path).convert('L').resize((self.size,self.size), Image.BILINEAR)
        B = Image.open(b_path).convert('RGB').resize((self.size,self.size), Image.NEAREST)
        A = (np.array(A, dtype=np.float32)/255.0 - 0.5)/0.5
        B = (np.array(B, dtype=np.float32)/255.0 - 0.5)/0.5
        A = torch.from_numpy(A)[None, ...]
        B = torch.from_numpy(B).permute(2,0,1)
        return A, B

# -------- pix2pix models --------
class UNetBlock(nn.Module):
    def __init__(self, in_ch, out_ch, down=True, use_dropout=False):
        super().__init__()
        if down:
            self.net = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(0.2, inplace=True),
            )
        else:
            seq = [
                nn.ConvTranspose2d(in_ch, out_ch, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
            ]
            if use_dropout: seq.append(nn.Dropout(0.5))
            self.net = nn.Sequential(*seq)
    def forward(self, x): return self.net(x)

class GeneratorUNet(nn.Module):
    def __init__(self, in_ch=1, out_ch=3, base=64):
        super().__init__()
        self.d1 = nn.Sequential(nn.Conv2d(in_ch, base, 4, 2, 1), nn.LeakyReLU(0.2, inplace=True))
        self.d2 = UNetBlock(base, base*2, down=True)
        self.d3 = UNetBlock(base*2, base*4, down=True)
        self.d4 = UNetBlock(base*4, base*8, down=True)
        self.d5 = UNetBlock(base*8, base*8, down=True)
        self.d6 = UNetBlock(base*8, base*8, down=True)
        self.d7 = UNetBlock(base*8, base*8, down=True)
        self.d8 = nn.Sequential(nn.Conv2d(base*8, base*8, 4, 2, 1), nn.ReLU(inplace=True))
        self.u1 = UNetBlock(base*8, base*8, down=False, use_dropout=True)
        self.u2 = UNetBlock(base*16, base*8, down=False, use_dropout=True)
        self.u3 = UNetBlock(base*16, base*8, down=False, use_dropout=True)
        self.u4 = UNetBlock(base*16, base*8, down=False)
        self.u5 = UNetBlock(base*16, base*4, down=False)
        self.u6 = UNetBlock(base*8, base*2, down=False)
        self.u7 = UNetBlock(base*4, base, down=False)
        self.u8 = nn.Sequential(nn.ConvTranspose2d(base*2, out_ch, 4, 2, 1), nn.Tanh())
    def forward(self, x):
        d1 = self.d1(x); d2 = self.d2(d1); d3 = self.d3(d2); d4 = self.d4(d3)
        d5 = self.d5(d4); d6 = self.d6(d5); d7 = self.d7(d6); d8 = self.d8(d7)
        u1 = self.u1(d8)
        u2 = self.u2(torch.cat([u1, d7], 1))
        u3 = self.u3(torch.cat([u2, d6], 1))
        u4 = self.u4(torch.cat([u3, d5], 1))
        u5 = self.u5(torch.cat([u4, d4], 1))
        u6 = self.u6(torch.cat([u5, d3], 1))
        u7 = self.u7(torch.cat([u6, d2], 1))
        out = self.u8(torch.cat([u7, d1], 1))
        return out

class PatchDiscriminator(nn.Module):
    def __init__(self, in_ch=4, base=64):
        super().__init__()
        def C(ic, oc, norm=True):
            seq = [nn.Conv2d(ic, oc, 4, 2, 1, bias=False)]
            if norm: seq.append(nn.BatchNorm2d(oc))
            seq.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*seq)
        self.net = nn.Sequential(
            C(in_ch, base, norm=False),
            C(base, base*2),
            C(base*2, base*4),
            nn.Conv2d(base*4, 1, 4, 1, 1)
        )
    def forward(self, x): return self.net(x)

# -------- data loaders --------
train_pairs = list_pairs("train")
val_pairs   = list_pairs("val")
test_pairs  = list_pairs("test", allow_empty=True)   # אופציונלי כרגע

print(f"Train: {len(train_pairs)} | Val: {len(val_pairs)} | Test (optional): {len(test_pairs)}")

train_dl = DataLoader(PairDataset(train_pairs, IMG_SIZE), batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, drop_last=True)
val_dl   = DataLoader(PairDataset(val_pairs,   IMG_SIZE), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dl  = DataLoader(PairDataset(test_pairs,  IMG_SIZE), batch_size=BATCH_SIZE, shuffle=False, num_workers=2) if test_pairs else None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = GeneratorUNet(1,3).to(device)
D = PatchDiscriminator(4).to(device)
optG = torch.optim.Adam(G.parameters(), lr=LR_G, betas=(0.5,0.999))
optD = torch.optim.Adam(D.parameters(), lr=LR_D, betas=(0.5,0.999))
bce = nn.BCEWithLogitsLoss()
l1  = nn.L1Loss()

# -------- helpers: external dataset versioning (robust) --------
def write_metadata(dirpath, username, slug, title=None, license_name="CC0-1.0"):
    os.makedirs(dirpath, exist_ok=True)
    meta = {"title": title or slug.replace("-", " ").title(),
            "id": f"{username}/{slug}",
            "licenses": [{"name": license_name}]}
    with open(os.path.join(dirpath, "dataset-metadata.json"), "w") as f:
        json.dump(meta, f, indent=2)

def _run(cmd):
    print(f"$ {cmd}")
    rc = os.system(cmd)
    if rc != 0:
        print(f"[WARN] Return code={rc} מהפקודה: {cmd}")
    return rc

def ensure_created_or_versioned(dirpath, username, slug, message):
    # ודאי שיש קבצים (חוץ מה-metadata) לפני יצירה/גרסה
    other_files = [p for p in glob.glob(os.path.join(dirpath, "*"))
                   if os.path.basename(p) != "dataset-metadata.json"]
    if not other_files:
        raise RuntimeError(f"'{dirpath}' ריק. הוסיפי קובץ אחד לפחות לפני create/version.")

    write_metadata(dirpath, username, slug)

    # 1) נסה CREATE (בלי -o):
    rc_create = _run(f'kaggle datasets create -p "{dirpath}" -r zip')
    # אם כבר קיים או נכשל, נמשיך ל- version
    # 2) נסה VERSION (לגרס). קודם בלי -r (עוקף את באג ה-'type'):
    rc_version = _run(f'kaggle datasets version -p "{dirpath}" -m "{message}"')
    if rc_version != 0:
        # נסה שוב עם -r zip (לחלק מהחשבונות זה דווקא עובד)
        rc_version = _run(f'kaggle datasets version -p "{dirpath}" -m "{message}" -r zip')

    if rc_create != 0 and rc_version != 0:
        raise RuntimeError(
            "נכשל ליצור/לגרס את הדאטהסט. אם את רואה 403, סביר שהדאטהסט עדיין לא נוצר או שאין הרשאה ליצור."
        )
    print(f"[OK] dataset {username}/{slug} עודכן/נוצר.")

# -------- resume: try to pull latest ckpt from external dataset --------
def download_ckpts_if_any(username, slug, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    rc = _run(f'kaggle datasets download -d {username}/{slug} -p "{target_dir}" -q --unzip')
    if rc != 0:
        print(f"[INFO] לא נמצא {username}/{slug} להורדה (כנראה עדיין לא נוצר). נתחיל מאפס.")

RESUME_DIR = f"{WORK_DIR}/RESUME"
download_latest_pair_only(USERNAME, CHECKPOINTS_DATASET_SLUG, RESUME_DIR)
# download_ckpts_if_any(USERNAME, CHECKPOINTS_DATASET_SLUG, RESUME_DIR)

def find_latest_epoch(ckpt_dir):
    ckpts = glob.glob(os.path.join(ckpt_dir, "**", "G_epoch_*.pt"), recursive=True)
    best_e = 0
    r = re.compile(r"G_epoch_(\d+)\.pt$")
    for p in ckpts:
        m = r.search(os.path.basename(p))
        if m: best_e = max(best_e, int(m.group(1)))
    return best_e

latest_e = find_latest_epoch(RESUME_DIR)
if latest_e > 0:
    print(f"Resuming from epoch {latest_e}")
    G.load_state_dict(torch.load(os.path.join(RESUME_DIR, f"G_epoch_{latest_e:03d}.pt"), map_location=device))
    D.load_state_dict(torch.load(os.path.join(RESUME_DIR, f"D_epoch_{latest_e:03d}.pt"), map_location=device))
    start_epoch = latest_e + 1
else:
    start_epoch = 1

# -------- visuals/history to outputs dataset --------
history = []
def denorm(x): return (x.clamp(-1,1)+1)*0.5

def save_sample_grid(A,B,F,out_path,nmax=8):
    A = denorm(A[:nmax]).cpu(); B = denorm(B[:nmax]).cpu(); F = denorm(F[:nmax]).cpu()
    A3 = A.repeat(1,3,1,1)
    rows = []
    for i in range(min(nmax, A3.shape[0])):
        rows.append(torch.cat([A3[i], F[i], B[i]], dim=2))
    grid = torch.cat(rows, dim=1)
    img = (grid.permute(1,2,0).numpy()*255).astype(np.uint8)
    Image.fromarray(img).save(out_path)

def export_ckpt_version(epoch, step_msg):
    # always write fresh ckpts then create/version external dataset
    os.makedirs(CKPT_LOCAL_DIR, exist_ok=True)
    torch.save(G.state_dict(), f"{CKPT_LOCAL_DIR}/G_epoch_{epoch:03d}.pt")
    torch.save(D.state_dict(), f"{CKPT_LOCAL_DIR}/D_epoch_{epoch:03d}.pt")
    ensure_created_or_versioned(CKPT_LOCAL_DIR, USERNAME, CHECKPOINTS_DATASET_SLUG, message=step_msg)

def export_outputs_version(epoch, note):
    os.makedirs(f"{OUT_LOCAL_DIR}/samples", exist_ok=True)
    # one visual sample from val
    A, B = next(iter(val_dl)); A, B = A.to(device), B.to(device)
    with torch.no_grad(): F = G(A)
    save_sample_grid(A,B,F, f"{OUT_LOCAL_DIR}/samples/epoch_{epoch:03d}.png")
    with open(f"{OUT_LOCAL_DIR}/training_history.json", "w") as f:
        json.dump(history, f, indent=2)
    ensure_created_or_versioned(OUT_LOCAL_DIR, USERNAME, OUTPUTS_DATASET_SLUG, message=note)

# -------- training loop with frequent external persistence --------
global_step = 0
for epoch in range(start_epoch, MAX_EPOCHS+1):
    G.train(); D.train()
    g_run=d_run=l1_run=0.0
    for A,B in tqdm(train_dl, desc=f"Epoch {epoch}"):
        A,B = A.to(device), B.to(device)

        # D
        with torch.no_grad(): F_fake = G(A)
        real_logits = D(torch.cat([A,B],1))
        fake_logits = D(torch.cat([A,F_fake.detach()],1))
        d_loss = bce(real_logits, torch.ones_like(real_logits)) + \
                 bce(fake_logits, torch.zeros_like(fake_logits))
        optD.zero_grad(); d_loss.backward(); optD.step()

        # G
        F_fake = G(A)
        adv = bce(D(torch.cat([A,F_fake],1)), torch.ones_like(fake_logits))
        l1_loss = l1(F_fake, B)*LAMBDA_L1
        g_loss = adv + l1_loss
        optG.zero_grad(); g_loss.backward(); optG.step()

        g_run += g_loss.item(); d_run += d_loss.item(); l1_run += l1_loss.item()
        global_step += 1

        # frequent persistence of checkpoints to external dataset
        if global_step % EXPORT_CKPT_EVERY_STEPS == 0:
            export_ckpt_version(epoch, step_msg=f"epoch {epoch} step {global_step}")

    n = len(train_dl)
    epoch_stats = {"epoch": epoch, "g": g_run/n, "d": d_run/n, "l1": l1_run/n}
    history.append(epoch_stats)

    # export outputs (samples+history) every few epochs
    if epoch % EXPORT_SAMPLES_EVERY_E == 0 or epoch == start_epoch:
        export_outputs_version(epoch, note=f"epoch {epoch}")

# final exports
export_ckpt_version(epoch, step_msg=f"final epoch {epoch}")
export_outputs_version(epoch, note=f"final epoch {epoch}")
print("Done.")

Kaggle API 1.7.4.5
Train: 2040 | Val: 360 | Test (optional): 0
Dataset URL: https://www.kaggle.com/datasets/yealsh/edgeseg-pix2pix-ckpts
License(s): CC0-1.0
Dataset URL: https://www.kaggle.com/datasets/yealsh/edgeseg-pix2pix-ckpts
License(s): CC0-1.0
[OK] הורדתי רק את ['G_epoch_182.pt', 'D_epoch_182.pt']
Resuming from epoch 182


Epoch 183:   0%|          | 0/255 [00:00<?, ?it/s]

$ kaggle datasets create -p "/kaggle/working/CKPTS" -r zip
Starting upload for file G_epoch_183.pt


100%|██████████| 208M/208M [00:08<00:00, 25.8MB/s]


Upload successful: G_epoch_183.pt (208MB)
Starting upload for file D_epoch_183.pt


...

Upload successful: samples.zip (7MB)
Starting upload for file training_history.json


100%|██████████| 2.00k/2.00k [00:00<00:00, 2.59kB/s]


Upload successful: training_history.json (2KB)
Dataset creation error: The requested title "Edgeseg Pix2Pix Outputs" is already in use by a dataset. Please choose another title.
$ kaggle datasets version -p "/kaggle/working/EXPORT" -m "final epoch 200"
Skipping folder: samples; use '--dir-mode' to upload folders
Starting upload for file training_history.json


100%|██████████| 2.00k/2.00k [00:00<00:00, 2.63kB/s]


Upload successful: training_history.json (2KB)
Dataset version is being created. Please check progress at https://www.kaggle.com/datasets/yealsh/edgeseg-pix2pix-outputs
[OK] dataset yealsh/edgeseg-pix2pix-outputs עודכן/נוצר.
Done.


In [4]:
import os, glob, torch, numpy as np
from PIL import Image
import torch.nn as nn

# --- הגדרות בסיס ---
IMG_SIZE = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- פונקציית denorm ---
def denorm(x):
    return (x.clamp(-1,1) + 1) * 0.5

# --- הגדרת הארכיטקטורה של ה-G ---
class UNetBlock(nn.Module):
    def __init__(self, in_ch, out_ch, down=True, use_dropout=False):
        super().__init__()
        if down:
            self.net = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(0.2, inplace=True),
            )
        else:
            seq = [
                nn.ConvTranspose2d(in_ch, out_ch, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
            ]
            if use_dropout:
                seq.append(nn.Dropout(0.5))
            self.net = nn.Sequential(*seq)
    def forward(self, x): return self.net(x)

class GeneratorUNet(nn.Module):
    def __init__(self, in_ch=1, out_ch=3, base=64):
        super().__init__()
        self.d1 = nn.Sequential(nn.Conv2d(in_ch, base, 4, 2, 1), nn.LeakyReLU(0.2, inplace=True))
        self.d2 = UNetBlock(base, base*2)
        self.d3 = UNetBlock(base*2, base*4)
        self.d4 = UNetBlock(base*4, base*8)
        self.d5 = UNetBlock(base*8, base*8)
        self.d6 = UNetBlock(base*8, base*8)
        self.d7 = UNetBlock(base*8, base*8)
        self.d8 = nn.Sequential(nn.Conv2d(base*8, base*8, 4, 2, 1), nn.ReLU(inplace=True))

        self.u1 = UNetBlock(base*8, base*8, down=False, use_dropout=True)
        self.u2 = UNetBlock(base*16, base*8, down=False, use_dropout=True)
        self.u3 = UNetBlock(base*16, base*8, down=False, use_dropout=True)
        self.u4 = UNetBlock(base*16, base*8, down=False)
        self.u5 = UNetBlock(base*16, base*4, down=False)
        self.u6 = UNetBlock(base*8, base*2, down=False)
        self.u7 = UNetBlock(base*4, base, down=False)
        self.u8 = nn.Sequential(nn.ConvTranspose2d(base*2, out_ch, 4, 2, 1), nn.Tanh())

    def forward(self, x):
        d1 = self.d1(x); d2 = self.d2(d1); d3 = self.d3(d2); d4 = self.d4(d3)
        d5 = self.d5(d4); d6 = self.d6(d5); d7 = self.d7(d6); d8 = self.d8(d7)
        u1 = self.u1(d8)
        u2 = self.u2(torch.cat([u1, d7], 1))
        u3 = self.u3(torch.cat([u2, d6], 1))
        u4 = self.u4(torch.cat([u3, d5], 1))
        u5 = self.u5(torch.cat([u4, d4], 1))
        u6 = self.u6(torch.cat([u5, d3], 1))
        u7 = self.u7(torch.cat([u6, d2], 1))
        return self.u8(torch.cat([u7, d1], 1))

# --- טעינת הצ'קפוינט האחרון מהתיקייה ---
RESUME_DIR = "/kaggle/working/RESUME"
ckpts = sorted(glob.glob(os.path.join(RESUME_DIR, "G_epoch_*.pt")))
assert ckpts, "לא נמצאו קבצי G_epoch_xxx.pt בתיקיית RESUME!"
last_ckpt = ckpts[-1]
print("Loading last checkpoint:", last_ckpt)

G = GeneratorUNet(1,3).to(device)
G.load_state_dict(torch.load(last_ckpt, map_location=device))
G.eval()
print("✅ Generator loaded and ready for inference")

Loading last checkpoint: /kaggle/working/RESUME/G_epoch_182.pt
✅ Generator loaded and ready for inference


In [5]:
# ===== Inference: Folder =====
# תיקיית קלט עם קבצי *_edges.* (png/jpg וכו')
IN_DIR  = "/kaggle/input/test-pix2pix/test_edges"
OUT_DIR = "/kaggle/working/preds"
os.makedirs(OUT_DIR, exist_ok=True)

edge_files = []
for ext in ("png","jpg","jpeg","bmp","tif","tiff"):
    edge_files += glob.glob(os.path.join(IN_DIR, f"*._edges.{ext}".replace("._",".*")))  # למקרה של סיומות שונות
    edge_files += glob.glob(os.path.join(IN_DIR, f"*_edges.{ext}"))

# fallback: אם אין התאמות עם התבניות למעלה, ניקח כל קובץ בתיקייה
if not edge_files:
    for ext in ("png","jpg","jpeg","bmp","tif","tiff"):
        edge_files += glob.glob(os.path.join(IN_DIR, f"*.{ext}"))

print("Found", len(edge_files), "files")

for p in edge_files:
    name = os.path.splitext(os.path.basename(p))[0]
    outp = os.path.join(OUT_DIR, f"pred_{name}.png")

    img = Image.open(p).convert('L').resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
    arr = (np.array(img).astype(np.float32)/255.0 - 0.5)/0.5
    A   = torch.from_numpy(arr)[None,None,...].to(device)

    with torch.no_grad():
        pred = G(A)

    pred_img = (denorm(pred)[0].permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
    Image.fromarray(pred_img).save(outp)

print("Saved to:", OUT_DIR)

Found 10 files
Saved to: /kaggle/working/preds


In [6]:
!ls -lh /kaggle/working/preds | head

total 944K
-rw-r--r-- 1 root root 110K Oct 26 23:32 pred_train_normal_00028_edges.png
-rw-r--r-- 1 root root  95K Oct 26 23:32 pred_train_normal_13926_edges.png
-rw-r--r-- 1 root root 113K Oct 26 23:32 pred_train_normal_14432_edges.png
-rw-r--r-- 1 root root  90K Oct 26 23:32 pred_train_normal_16090_edges.png
-rw-r--r-- 1 root root  95K Oct 26 23:32 pred_train_normal_18312_edges.png
-rw-r--r-- 1 root root  80K Oct 26 23:32 pred_train_normal_18355_edges.png
-rw-r--r-- 1 root root  88K Oct 26 23:32 pred_train_normal_20930_edges.png
-rw-r--r-- 1 root root  80K Oct 26 23:32 pred_train_normal_21862_edges.png
-rw-r--r-- 1 root root  79K Oct 26 23:32 pred_train_normal_22512_edges.png


In [7]:
import os
from IPython.display import Image, display
from PIL import Image as PILImage
import matplotlib.pyplot as plt

pred_dir = "/kaggle/working/preds"
files = sorted([os.path.join(pred_dir, f) for f in os.listdir(pred_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

print(f"נמצאו {len(files)} תמונות בתיקייה.")

# הגבלה להצגה של עד 30 תמונות בבת אחת (אפשר לשנות)
max_show = 30

cols = 5
rows = min((len(files) + cols - 1) // cols, 6)
fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))

for i, ax in enumerate(axes.flat):
    if i < len(files) and i < max_show:
        img = PILImage.open(files[i])
        ax.imshow(img)
        ax.set_title(os.path.basename(files[i]), fontsize=8)
        ax.axis("off")
    else:
        ax.axis("off")

plt.tight_layout()
plt.show()

נמצאו 10 תמונות בתיקייה.


In [8]:
!zip -r /kaggle/working/preds.zip /kaggle/working/preds

  adding: kaggle/working/preds/ (stored 0%)
  adding: kaggle/working/preds/pred_train_normal_22512_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_18355_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_23519_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_00028_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_18312_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_16090_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_20930_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_14432_edges.png (deflated 0%)
  adding: kaggle/working/preds/pred_train_normal_21862_edges.png (deflated 1%)
  adding: kaggle/working/preds/pred_train_normal_13926_edges.png (deflated 0%)
